In [1]:
from data import MaskDataset, MaskDataLoader

import os
import logging

import torchvision
from torchvision import transforms, models
from torchvision.transforms.functional import to_pil_image
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageFile
from tqdm.notebook import tqdm

ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline

MODEL_SAVE_PATH = '/opt/ml/checkpoints'

In [3]:
# Import error
# %conda install -c conda-forge ipywidgets

In [4]:
class BaseModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=7, stride=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.25)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout2(x)

        x = self.avgpool(x)
        x = x.view(-1, 128)
        return self.fc(x)

In [5]:
pretrained_list = timm.list_models(pretrained=True)
# print(pretrained_list)
for model in pretrained_list:
    if 'swin' in model:
        print(model)
    if 'efficient' in model:
        print(model)

efficientnet_b0
efficientnet_b1
efficientnet_b1_pruned
efficientnet_b2
efficientnet_b2_pruned
efficientnet_b3
efficientnet_b3_pruned
efficientnet_b4
efficientnet_el
efficientnet_el_pruned
efficientnet_em
efficientnet_es
efficientnet_es_pruned
efficientnet_lite0
efficientnetv2_rw_m
efficientnetv2_rw_s
efficientnetv2_rw_t
gc_efficientnetv2_rw_t
swin_base_patch4_window7_224
swin_base_patch4_window7_224_in22k
swin_base_patch4_window12_384
swin_base_patch4_window12_384_in22k
swin_large_patch4_window7_224
swin_large_patch4_window7_224_in22k
swin_large_patch4_window12_384
swin_large_patch4_window12_384_in22k
swin_small_patch4_window7_224
swin_tiny_patch4_window7_224
tf_efficientnet_b0
tf_efficientnet_b0_ap
tf_efficientnet_b0_ns
tf_efficientnet_b1
tf_efficientnet_b1_ap
tf_efficientnet_b1_ns
tf_efficientnet_b2
tf_efficientnet_b2_ap
tf_efficientnet_b2_ns
tf_efficientnet_b3
tf_efficientnet_b3_ap
tf_efficientnet_b3_ns
tf_efficientnet_b4
tf_efficientnet_b4_ap
tf_efficientnet_b4_ns
tf_efficientnet_b

In [6]:
# target_model = models.resnext50_32x4d(pretrained=True)
target_model = models.efficientnet_b3(pretrianed=True)
# target_model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=18)
# target_model = BaseModel(18)
model_name = 'efficientnet_b3'

# print("네트워크 필요 입력 채널 개수", target_model.layers[0].weight.shape[1])
# print("네트워크 출력 채널 개수 (예측 class type 개수)", target_model.layers[-1].weight.shape[0])

print(target_model)

AttributeError: module 'torchvision.models' has no attribute 'efficientnet_b3'

In [6]:
print(os.getcwd())

/opt/ml/code


In [7]:
mean = (0.548, 0.504, 0.479)
std = (0.237, 0.247, 0.246)

transform = transforms.Compose([
                    # transforms.Resize((384,384)), 
                    transforms.RandomCrop((384,384)),
                    transforms.ToTensor(), 
                    transforms.Normalize(mean, std)
                ])
train_dataset = MaskDataset(transform, training=True)
test_dataset = MaskDataset(transform, training=False)

INFO:root:Converting train dataset...


18900it: 0it [00:00, ?it/s]

INFO:root:Converting test dataset...


12600it: 0it [00:00, ?it/s]

In [8]:
print(train_dataset[0])
print(test_dataset[0])

(tensor([[[ 0.6827,  0.6993,  0.6993,  ..., -1.5842, -1.4684, -1.3360],
         [ 0.6827,  0.6993,  0.6993,  ..., -1.5676, -1.4022, -1.2367],
         [ 0.6827,  0.6827,  0.6827,  ..., -1.8158, -1.7993, -1.6007],
         ...,
         [ 0.4842,  0.5007,  0.5007,  ...,  0.5338,  0.4180,  0.3683],
         [ 0.4842,  0.4842,  0.4842,  ...,  0.5173,  0.4511,  0.3849],
         [ 0.4676,  0.4676,  0.4676,  ...,  0.5338,  0.4676,  0.4014]],

        [[ 0.8015,  0.8173,  0.8173,  ..., -1.8817, -1.8817, -1.7706],
         [ 0.8015,  0.8173,  0.8173,  ..., -1.8182, -1.7388, -1.6277],
         [ 0.8015,  0.8015,  0.8015,  ..., -1.9452, -1.9452, -1.8023],
         ...,
         [-0.1829, -0.1670, -0.1670,  ..., -0.2147, -0.2623, -0.3099],
         [-0.1829, -0.1829, -0.1829,  ..., -0.1829, -0.2305, -0.2623],
         [-0.1988, -0.1988, -0.1988,  ..., -0.1670, -0.2147, -0.2464]],

        [[ 0.7150,  0.7310,  0.7310,  ..., -1.7718, -1.7240, -1.6124],
         [ 0.7150,  0.7310,  0.7310,  ..., -

In [9]:
print(train_dataset[0][0].size())
print(test_dataset[0][0].size())

torch.Size([3, 384, 384])
torch.Size([3, 384, 384])


In [10]:
# IMAGE_INDEX = 0
# fig, axes = plt.subplots(1, 2, figsize=(15,7))

# img, label = train_dataset[IMAGE_INDEX][0], train_dataset[IMAGE_INDEX][1]
# normalized_img = to_pil_image(img)
# origin_img = to_pil_image(std*img+mean)

# axes[0].imshow(normalized_img)
# axes[0].set_title('Normalized image')
# axes[0].axis('off')
# axes[1].imshow(origin_img)
# axes[1].set_title('Origin image')
# axes[1].axis('off')

# print(f'label: ', label)

In [11]:
# DataLoader
BATCH_SIZE = 64
train_dataloader = MaskDataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, validation_split=0.2)
val_dataloader = train_dataloader.split_validation()
# test_dataloader = MaskDataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
next(iter(train_dataloader))

[tensor([[[[ 1.1460,  1.1460,  1.1460,  ...,  1.0137,  1.0137,  0.9971],
           [ 1.1460,  1.1460,  1.1460,  ...,  1.0137,  0.9971,  0.9971],
           [ 1.1460,  1.1460,  1.1460,  ...,  0.9971,  0.9971,  0.9971],
           ...,
           [ 1.3611,  1.3611,  1.3115,  ..., -0.3432, -0.3432, -0.3266],
           [ 1.3611,  1.3611,  1.3115,  ..., -0.3597, -0.3432, -0.3266],
           [ 1.3611,  1.3611,  1.3115,  ..., -0.3597, -0.3597, -0.3266]],
 
          [[ 1.2778,  1.2778,  1.2778,  ...,  1.0872,  1.0872,  1.0714],
           [ 1.2778,  1.2778,  1.2778,  ...,  1.0872,  1.0714,  1.0714],
           [ 1.2778,  1.2778,  1.2778,  ...,  1.0714,  1.0714,  1.0714],
           ...,
           [ 1.5000,  1.5000,  1.4524,  ..., -0.1829, -0.1829, -0.1670],
           [ 1.5000,  1.5000,  1.4524,  ..., -0.1988, -0.1829, -0.1670],
           [ 1.5000,  1.5000,  1.4524,  ..., -0.1988, -0.1988, -0.1670]],
 
          [[ 1.2571,  1.2571,  1.2571,  ...,  1.0020,  1.0020,  0.9861],
           [ 

In [13]:
# Change last FC layer
# import math

# MASK_CLASS_NUM = 18
# in_features = target_model.fc.in_features
# target_model.fc = torch.nn.Linear(in_features=in_features, out_features=MASK_CLASS_NUM, bias=True)

# torch.nn.init.xavier_uniform_(target_model.fc.weight)
# stdv = 1. / math.sqrt(target_model.fc.weight.size(1))
# target_model.fc.bias.data.uniform_(-stdv, stdv)

# print("네트워크 필요 입력 채널 개수", target_model.conv1.weight.shape[1])
# print("네트워크 출력 채널 개수 (예측 class type 개수)", target_model.fc.weight.shape[0])

In [14]:
# Compile options
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"{device} is using!")

# resnext50.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 10

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train": train_dataloader, 
    "val": val_dataloader
}

cuda:0 is using!


In [15]:
# training
target_model.to(device)
# 저장 경로 생성
if not os.path.exists(f"{MODEL_SAVE_PATH}/{model_name}"):
    os.mkdir(f"{MODEL_SAVE_PATH}/{model_name}")
# freeze layers except last FC layer
# for param in target_model.parameters()[:-1]:
#     param.requires_grad = False

best_val_accuracy = 0.
best_val_loss = 999.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "val"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "val":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = criterion(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() #* images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    if phase == 'train':
        epoch_loss = running_loss / (len(dataloaders[phase].dataset))
        epoch_acc = running_acc / (len(dataloaders[phase].dataset))
    else:
        epoch_loss = running_loss / (len(dataloaders[phase].dataset))
        epoch_acc = running_acc / (len(dataloaders[phase].dataset))

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "val" and best_val_accuracy < epoch_acc: # phase가 val일 때, best accuracy 계산
      print("New best model for val accuracy! Saving the model...")
      path = os.path.join(MODEL_SAVE_PATH, f"{model_name}/{epoch:03}_acc_{epoch_acc:4.2%}.ckpt")
      torch.save(target_model.state_dict(), path)
      best_val_accuracy = epoch_acc
    if phase == "val" and best_val_loss > epoch_loss: # phase가 val일 때, best loss 계산
      print("New best model for val loss! Saving the model...")
      path = os.path.join(MODEL_SAVE_PATH, f"{model_name}/{epoch:03}_loss_{epoch_loss:4.2}.ckpt")
      torch.save(target_model.state_dict(), path)
      best_val_loss = epoch_loss



print("학습 종료!")
print(f"최고 accuracy : {best_val_accuracy}, 최고 낮은 loss : {best_val_loss}")

  0%|          | 0/237 [00:00<?, ?it/s]

현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.007, 평균 Accuracy : 0.696


  0%|          | 0/60 [00:00<?, ?it/s]

현재 epoch-0의 val-데이터 셋에서 평균 Loss : 0.000, 평균 Accuracy : 0.190
New best model for val accuracy! Saving the model...
New best model for val loss! Saving the model...


  0%|          | 0/237 [00:00<?, ?it/s]

...

현재 epoch-7의 train-데이터 셋에서 평균 Loss : 0.013, 평균 Accuracy : 0.511


  0%|          | 0/60 [00:00<?, ?it/s]

현재 epoch-7의 val-데이터 셋에서 평균 Loss : 0.004, 평균 Accuracy : 0.114


  0%|          | 0/237 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# # 모델 저장
# PATH = os.path.join('/opt/ml/checkpoints', 'resnext50_epochs10_acc32.pt')
# torch.save(target_model, PATH)
# # model = torch.load(PATH)
# # model.eval()

In [6]:
import time

print(time.time())

1646311680.461598


## Class 별 개수 세기

![](https://s3-us-west-2.amazonaws.com/aistages-prod-server-public/app/Users/00001204/files/d7f11b5e-2752-4aff-9d0f-6f46da8a3b19..png)

In [2]:
from glob import glob
import pandas as pd

total_train_images = glob('/opt/ml/input/data/train/images/*/*') # 모든 train 이미지에 대한 경로를 리스트로 받아옵니다.

DATA_PATH = '/opt/ml/input/data/train/train.csv'

def age_group(x): # 김준재님 코드 참고
    return min(2, x // 30)

def preprocess_train_dataframe(total_train_images):

    new_train_df = pd.DataFrame(columns={"id", "gender", "race", "age", "mask", "img_path", "path"})
    total_id, total_gender, total_race, total_age, total_img_path, total_mask, total_folder = [], [], [], [], [], [], []

    for img_path in total_train_images:

        split_list = img_path.split("/")
        file_name = split_list[-1]
        path = split_list[-2]

        path_split = path.split("_")
        id_ = path_split[0]
        gender = 0 if path_split[1] == "male" else 1
        race = path_split[2]
        age = int(path_split[3])
        
        if "normal" in file_name:
            mask = 2
        elif "incorrect" in file_name:
            mask = 1
        else:
            mask = 0

        total_id.append(id_)
        total_gender.append(gender)
        total_race.append(race)
        total_age.append(age)
        total_mask.append(mask)
        total_img_path.append(img_path)
        total_folder.append(path)   

    new_train_df['id'] = total_id
    new_train_df['gender'] = total_gender
    new_train_df['race'] = total_race
    new_train_df['age'] = total_age
    new_train_df['mask'] = total_mask
    new_train_df['img_path'] = total_img_path
    new_train_df['path'] = total_folder
    
    # age group 생성
    new_train_df['age_group'] = new_train_df['age'].apply(lambda x : age_group(x))

    # label 생성 - 신규범님 코드 참고
    new_train_df['label'] = new_train_df['mask'] * 6 + new_train_df['gender']*3 + new_train_df['age_group']
    return new_train_df.sort_values(by='id').reset_index(drop=True)

new_train_df = preprocess_train_dataframe(DATA_PATH)

IndexError: list index out of range

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(1, 1, figsize=(15, 7))
ax = sns.countplot(x="label", data=new_train_df, ax=ax)
for p in ax.patches:
        ax.annotate('{}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+30))